In [ ]:
from bs4 import BeautifulSoup as bs
import lxml
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', sys.maxsize)
pd.set_option('display.max_colwidth', None)

In [ ]:
with open("UC 3/etl_generated/R20-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml", "r") as file:
    handler20 = file.read()

with open("UC 3/etl_generated/R21-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml", "r") as file:
    handler21 = file.read()

soup20 = bs(handler20, "lxml-xml")
list_of_all_ids20 = soup20.find_all("SHORT-NAME")
soup21 = bs(handler21, "lxml-xml")
list_of_all_ids21 = soup21.find_all("SHORT-NAME")

list_of_all_ids20 = [id.getText() for id in list_of_all_ids20]
list_of_all_ids21 = [id.getText() for id in list_of_all_ids21]

df20 = pd.DataFrame({"ids20": list_of_all_ids20})
df21 = pd.DataFrame({"ids21": list_of_all_ids21})
df_merged = pd.merge(df20, df21, how = "outer", left_on = "ids20", right_on = "ids21")
df_merged["type"] = "unchanged"
df_merged["type"][df_merged.ids21.isna()] = "delete"
df_merged["type"][df_merged.ids20.isna()] = "new"
df_merged.head()

# New Part: Change Detection

In [ ]:
import xml.etree.ElementTree as ET
from lxml import etree

In [413]:
tree = etree.parse('UC 3/etl_generated/R21-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml')
ids20simple = tree.xpath("//x:REQUIREMENT[count(.//x:L-1) = 1]/x:TRACE/x:SHORT-NAME/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})
text20simple = tree.xpath("//x:REQUIREMENT[count(.//x:L-1) = 1]//x:L-1/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})

In [416]:
text20simple

['\n               ',
 ' shall access the location of the API of all used underlying ',
 ' for link time configuration by a set of function pointers for each ',
 '.',
 'The header file ',
 ' only contains extern declarations of constants, global data and services that are specified in ',
 '.',
 '\n               ',
 ' shall include the header file ',
 ' if Bus Mirroring is enabled (see ',
 ').',
 '\n               ',
 ' shall avoid direct access to hardware specific communication buffers and shall access it exclusively via ',
 ' interface services.',
 'Definition of ',
 ': The ',
 ' shall be a handle referencing a logical ',
 ' of the CAN Controller mailbox.',
 'The ',
 ' shall enable ',
 ' to use ',
 ' or a ',
 ' reception method of the referenced reception unit and to indicate a Received ',
 ' to a target upper layer module.',
 'If the ',
 ' references a reception unit configured for ',
 ', software filtering shall be enabled in ',
 '.',
 'If multiple ',
 ' are used, each ',
 ' shall

In [414]:
data20simple = pd.DataFrame({"ids20": ids20simple, "text20": text20simple})
data20simple = data20simple.loc[data20simple["ids20"].str[:4] != "ECUC"]

ValueError: All arrays must be of the same length

In [ ]:
data20simple

In [406]:
tree = etree.parse('UC 3/etl_generated/R22-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml')
ids21simple = tree.xpath("//x:REQUIREMENT[count(.//x:L-1) = 1]/x:TRACE/x:SHORT-NAME/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})
text21simple = tree.xpath("//x:REQUIREMENT[count(.//x:L-1) = 1]//x:L-1/text()", namespaces={'x': 'http://autosar.org/schema/r4.0'})

In [407]:
data21simple = pd.DataFrame({"ids21": ids21simple, "text21": text21simple})
data21simple = data21simple.loc[data21simple["ids21"].str[:4] != "ECUC"]

In [ ]:
data21simple

In [408]:
datamergedsimple = pd.merge(data20simple, data21simple, how="outer", left_on="ids20", right_on="ids21")

In [409]:
datamergedsimple["type"] = "unchanged"
datamergedsimple["type"].loc[datamergedsimple.ids21.isna()] = "delete"
datamergedsimple["type"].loc[datamergedsimple.ids20.isna()] = "new"

In [ ]:
datamergedsimple.loc[0, "text21"] = "asdf"

In [410]:
datamergedsimple.loc[(datamergedsimple["type"] == "unchanged") & (datamergedsimple["text20"] != datamergedsimple["text21"]), "type"] = "changed"

In [412]:
datamergedsimple.loc[datamergedsimple["ids20"] == "SWS_CANIF_00378"]

,ids20,text20,ids21,text21,type
0,SWS_CANIF_00378,\n,SWS_CANIF_00378,\n,unchanged


# Get different tag formats

In [ ]:
tree = etree.parse('UC 3/etl_generated/R20-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml')
tree.xpath("//x:TRACE", namespaces={'x': 'http://autosar.org/schema/r4.0'})

In [ ]:
[elem for elem in tree.iter() if elem.tag == "{http://autosar.org/schema/r4.0}TRACE"]

In [ ]:
for elem in tree.iter():
    if elem.tag == "{http://autosar.org/schema/r4.0}TRACE":
        print(elem.tag)

In [ ]:
tree.findall("{http://autosar.org/schema/r4.0}REQUIREMENTS")

In [ ]:
def getStructures(tablepath):
    tree = etree.parse(tablepath)

    ls = []
    struct = []
    for tag in [elem.tag[32:] for elem in tree.xpath("//*", namespaces={'x': 'http://autosar.org/schema/r4.0'})][1:]:
        if tag == "REQUIREMENT":
            ls.append(struct)
            struct = [tag]
        elif tag != "REQUIREMENT": 
                struct.append(tag)
    ls = ls[1:]

    set = []
    for x in ls:
        if x not in set:
            set.append(x)

    counts = []
    for struct in set:
        counts.append(ls.count(struct))

    rel = []
    for count in counts:
        rel.append(count/sum(counts))

    structtable = pd.DataFrame({"structure": set, "count": counts, "relative": rel})

    return structtable

In [ ]:
s20 = getStructures('UC 3/etl_generated/R20-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml')
s20.loc[s20.relative > 0.15]

In [ ]:
s21 = getStructures('UC 3/etl_generated/R21-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml')
s21.loc[s21.relative > 0.15]

In [ ]:
s22 = getStructures('UC 3/etl_generated/R22-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml')
s22.loc[s22.relative > 0.15]

In [400]:
s20 = getStructures('UC 3/etl_generated/R20-11/SWS_CANDriver_011/zz_generated/AUTOSAR_SWS_CANDriver_resolved.arxml')

# Filter StructureTable by number of L-1 tags
structs = []
counts = []
relatives = []
for index, row in s20.iterrows():
    if row.structure.count("L-1") == 1:
        structs.append(row.structure)
        counts.append(row[1])
        relatives.append(row.relative)
df = pd.DataFrame({"structure": structs, "count": counts, "relative": relatives})

In [ ]:
sum(df.relative)

In [402]:
df

,structure,count,relative
0,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, P, L-1]",53,0.193431
1,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, P, L-1]",90,0.328467
2,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, TRACE-REF, P, L-1]",12,0.043796
3,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, P, L-1]",2,0.007299
4,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, TRACE-REF, TRACE-REF, P, L-1]",5,0.018248
5,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, P, L-1]",1,0.003650
6,"[REQUIREMENT, TRACE, SHORT-NAME, TRACE-REFS, P, L-1, XREF, REFERRABLE-REF]",52,0.189781
7,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, P, L-1]",1,0.003650


In [401]:
s20

,structure,count,relative
0,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, P, L-1]",53,0.193431
1,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, P, L-1]",90,0.328467
2,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, TRACE-REF, P, L-1]",12,0.043796
3,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, TRACE-REF, P, L-1]",2,0.007299
4,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, TRACE-REF, TRACE-REF, P, L-1]",5,0.018248
5,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, LIST, ITEM, P, L-1, ITEM, P, L-1, ITEM, P, L-1]",1,0.003650
6,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, P, L-1, P, L-1, P, L-1, P, L-1]",1,0.003650
7,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, P, L-1, LIST, ITEM, P, L-1, P, L-1, LIST, ITEM, P, L-1]",1,0.003650
8,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, TRACE-REF, P, L-1, P, L-1]",1,0.003650
9,"[REQUIREMENT, TRACE, SHORT-NAME, LONG-NAME, L-4, TRACE-REFS, P, L-1, P, L-1]",3,0.010949
